# Wielowymiarowy filtr Kalmana

Wielowymiarowy filtr Kalmana składa się z tych samych pięciu równań co jednowymiarowy filtr, lecz w formie macierzowej - umożliwia to obliczanie stanu składającego się z wielu wartości w jednym cyklu filtra.

Równania filtru Kalmana w formie macierzowej wyglądają następująco:

## Równanie ekstrapolacji stanu

Równanie ekstrapolacji stanu umożliwia estymację kolejnego stanu filtra na podstawie poprzedniego stanu oraz nowych danych.

$$ \bold{\hat{x}_{n+1,n}} = \bold{F\hat{x}_{n,n}} + \bold{Gu_n} $$

Gdzie:

* $\bold{\hat{x}_{n+1,n}}$ to wektor przewidywanego stanu systemu w momencie $n + 1$
* $\bold{F}$ to macierz przejścia stanu, określająca wpływ poprzedniego stanu na kolejny
* $\bold{\hat{x}_{n,n}}$ to wektor estymowanego stanu systemu w momencie $n$
* $\bold{u_n}$ to wektor wejściowy (aktualizacji), zawierający dane wejściowe do systemu (na przykład pomiary czujników)
* $\bold{G}$ to macierz kontrolna, określająca wpływ pomiarów na stan

Dla uproszczenia, poniższe przykłady będą na bazie czujników jednoosiowych - rozszerzenie ich do większej ilości osi jest trywialne, ponieważ wymaga tylko dodania współczynników dla kolejnych osi do macierzy.

Przykładowo, dla filtra który ma estymować pozycję na podstawie danych z akcelerometru jednowymiarowego, wektor stanu może wyglądać następująco:
$$ \bold{\hat{x}_n} = \begin{bmatrix}
    \hat{x}_n \\
    \hat{\dot{x}}_n \\
\end{bmatrix} $$

Ten wektor ma rozmiar $n_x \times 1 $, gdzie $n_x$ to ilość wartości w wektorze stanu

gdzie $\hat{x}_n$ to pozycja, a $\hat{\dot{x}}_n$ to prędkość w momencie $n$ i osi x.

Macierz przejścia stanu jest wyprowadzona na podstawie równań systemu. W naszym przypadku, równania będą następujące:

$$ \begin{cases}
    x_n = x_{n-1} + \dot{x}_{n-1} \Delta t + \frac{1}{2} \ddot{x}_{n-1} \Delta t^2 \\
    \dot{x}_n = \dot{x}_{n-1} + \ddot{x}_{n-1} \Delta t \\
\end{cases} $$
 
Przenosząc to na macierz przejścia stanu otrzymujemy

$$ \bold{F} = \begin{bmatrix}
    1 & \Delta t \\
    0 & 1 \\
\end{bmatrix} $$

Macierz przejścia stanu ma rozmiar $n_x \times n_x$, gdzie $n_x$ to ilość wartości w wektorze stanu

Wektor wejściowy zawiera nowe dane dla naszego systemu. W naszym przypadku będzie on jednoelementowy, ponieważ nasz przykładowy akcelerometr operuje w jednej osi

$$ \bold{u_n} = \begin{bmatrix} \ddot{x}_n \end{bmatrix} $$

Wektor wejściowy ma wielkość $n_u \times 1$, gdzie $n_u$ to ilość wartości w wektorze wejściowym

Macierz kontrolna określa w jaki sposób wartości wejściowe wpływają na stan. W naszym przypadku, zgodnie z wyżej wyprowadzonym układem równań, będzie wyglądać następująco:

$$ \bold{G} = \begin{bmatrix}
    0.5 \Delta t^2 \\
    \Delta t \\
\end{bmatrix} $$ 

Macierz kontrolna ma rozmiar $n_x \times n_u$, gdzie $n_x$ to długość wektora stanu a $n_u$ to długość wektora wejściowego

W celu sprawdzenia poprawności powyższych macierzy, możemy wyprowadzić układ równań naszego modelu z równania ekstrapolacji stanu

$$
\begin{bmatrix}
    \hat{x}_{n+1,n} \\
    \hat{\dot{x}}_{n+1,n} \\
\end{bmatrix} =
\begin{bmatrix}
    1 & \Delta t \\
    0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
    \hat{x}_{n,n} \\
    \hat{\dot{x}}_{n,n} \\
\end{bmatrix} +
\begin{bmatrix}
    0.5 \Delta t^2 \\
    \Delta t \\
\end{bmatrix}
\begin{bmatrix} \ddot{x}_n \end{bmatrix}
$$

Po uproszczeniu otrzymujemy 

$$
\begin{bmatrix}
    \hat{x}_{n+1,n} \\
    \hat{\dot{x}}_{n+1,n} \\
\end{bmatrix} =
\begin{bmatrix}
    \hat{x}_{n,n} + \Delta t \hat{\dot{x}}_{n,n} + 0.5 \Delta t^2 \ddot{x}_{n,n} \\
    \hat{\dot{x}}_{n,n} + \Delta t \ddot{x}_{n,n}
\end{bmatrix}
$$

co dowodzi poprawności macierzy.

## Równanie ekstrapolacji niepewności (kowariancji)

Równanie ekstrapolacji niepewności umożliwia estymację niepewności systemu na podstawie niepewności w poprzedniej iteracji filtra

$$ \bold{P_{n+1,n}} = \bold{F P_{n,n} F^T} + \bold{Q} $$

Gdzie:

* $\bold{P_{n+1,n}}$ to macierz kowariancji (niepewności) kolejnego stanu
* $\bold{F}$ to macierz przejścia stanu, opisana wyżej. $\bold{F^T}$ to transponowana macierz przejścia stanu.
* $\bold{P_{n,n}}$ to macierz kowariancji (niepewności) aktualnego stanu
* $\bold{Q}$ to macierz szumu procesu